In [1]:
import pandas as pd

# Load data from Excel files
carbon_emissions_df = pd.read_csv('dataset/carbon_emissions.csv')
cost_profiles_df = pd.read_csv('dataset/cost_profiles.csv')
demand_df = pd.read_csv('dataset/demand.csv')
fuels_df = pd.read_csv('dataset/fuels.csv')
vehicles_df = pd.read_csv('dataset/vehicles.csv')
vehicles_fuels_df = pd.read_csv('dataset/vehicles_fuels.csv')

In [2]:
carbon_emissions_df.head()

,Year,Carbon emission CO2/kg
0,2023,11677957
1,2024,10510161
2,2025,9459145
3,2026,8513230
4,2027,7661907


In [3]:
cost_profiles_df.head()

,End of Year,Resale Value %,Insurance Cost %,Maintenance Cost %
0,1,90,5,1
1,2,80,6,3
2,3,70,7,5
3,4,60,8,7
4,5,50,9,9


In [4]:
demand_df.head()

,Year,Size,Distance,Demand (km)
0,2023,S1,D1,869181
1,2023,S1,D2,2597094
2,2023,S1,D3,3292011
3,2023,S1,D4,414315
4,2023,S2,D1,995694


In [5]:
fuels_df.head()

,Fuel,Year,Emissions (CO2/unit_fuel),Cost ($/unit_fuel),Cost Uncertainty (±%)
0,B20,2023,3.04858,1.220845,0
1,B20,2024,3.04858,1.246802,2
2,B20,2025,3.04858,1.273310,4
3,B20,2026,3.04858,1.300382,6
4,B20,2027,3.04858,1.328030,8


In [6]:
# print(vehicles_df['ID'].nunique())
# print(vehicles_df.shape)
vehicles_df.head(200)

,ID,Vehicle,Size,Year,Cost ($),Yearly range (km),Distance
0,BEV_S1_2023,BEV,S1,2023,187000,102000,D1
1,BEV_S1_2024,BEV,S1,2024,177650,102000,D1
2,BEV_S1_2025,BEV,S1,2025,168767,102000,D1
3,BEV_S1_2026,BEV,S1,2026,160329,102000,D2
4,BEV_S1_2027,BEV,S1,2027,152312,102000,D2
...,...,...,...,...,...,...,...
187,LNG_S3_2034,LNG,S3,2034,209208,73000,D4
188,LNG_S3_2035,LNG,S3,2035,215484,73000,D4
189,LNG_S3_2036,LNG,S3,2036,221948,73000,D4
190,LNG_S3_2037,LNG,S3,2037,228607,73000,D4


In [7]:
pd.set_option('display.max_rows', 500)

In [8]:
# print(vehicles_fuels_df.groupby(['ID'])['Fuel'].nunique())
vehicles_fuels_df.head()

,ID,Fuel,Consumption (unit_fuel/km)
0,BEV_S1_2023,Electricity,0.893043
1,BEV_S1_2024,Electricity,0.893043
2,BEV_S1_2025,Electricity,0.893043
3,BEV_S1_2026,Electricity,0.893043
4,BEV_S1_2027,Electricity,0.868161


In [128]:
import gurobipy as gp
from gurobipy import GRB

# Create Gurobi model
model = gp.Model("FleetDecarbonization")

# Define sets
vehicles = vehicles_df['ID'].unique() #vehicles
years = list(range(2023, 2039)) #years
fuels = fuels_df['Fuel'].unique() #type of fuels
sizes = ["S1", "S2", "S3", "S4"]  #size buckets
distances = ["D1", "D2", "D3", "D4"]  #distance buckets


In [129]:
# Parameters
demand = {(y, s, d): demand_df[(demand_df['Year']==y)&(demand_df['Size']==s)&(demand_df['Distance']==d)]['Demand (km)'].values[0] for y in years for s in sizes for d in distances}
vehicle_cost = {v: vehicles_df[vehicles_df['ID']==v]["Cost ($)"].values[0] for v in vehicles}
vehicle_range = {v: vehicles_df[vehicles_df['ID']==v]["Yearly range (km)"].values[0] for v in vehicles}
vehicle_size = {v: vehicles_df[vehicles_df['ID']==v]["Size"].values[0] for v in vehicles}
vehicle_distance = {v: vehicles_df[vehicles_df['ID']==v]["Distance"].values[0] for v in vehicles}
fuel_set = {v: list(vehicles_fuels_df[vehicles_fuels_df['ID']==v]['Fuel'].unique()) for v in vehicles}
fuel_consumption = {(v, f): vehicles_fuels_df[(vehicles_fuels_df['ID']==v)&(vehicles_fuels_df['Fuel']==f)]['Consumption (unit_fuel/km)'].values[0] for v in vehicles for f in fuel_set[v]}
fuel_cost = {(y, f): fuels_df[(fuels_df['Year']==y)&(fuels_df['Fuel']==f)]["Cost ($/unit_fuel)"].values[0] for y in years for f in fuels}
fuel_emissions = {(y, f): fuels_df[(fuels_df['Fuel']==f)&(fuels_df['Year']==y)]["Emissions (CO2/unit_fuel)"].values[0] for f in fuels for y in years}
carbon_budget = {y: carbon_emissions_df[carbon_emissions_df['Year']==y]['Carbon emission CO2/kg'].values[0] for y in years}
resale_value = {1: 0.9, 2: 0.8, 3: 0.7, 4: 0.6, 5: 0.5, 6: 0.4, 7: 0.3, 8: 0.3, 9: 0.3, 10: 0.3}
insurance_cost = {1: 0.05, 2: 0.06, 3: 0.07, 4: 0.08, 5: 0.09, 6: 0.10, 7: 0.11, 8: 0.12, 9: 0.13, 10: 0.14}
maintenance_cost = {1: 0.01, 2: 0.03, 3: 0.05, 4: 0.07, 5: 0.09, 6: 0.11, 7: 0.13, 8: 0.15, 9: 0.17, 10: 0.19}
vehile_size_map = {(v, s): vehicles_df[(vehicles_df['ID']==v)&(vehicles_df['Size']==s)]['Size'].count() for v in vehicles for s in sizes}
distance_bucket_map = {"D1": 300, "D2": 400, "D3": 500, "D4": 600}
vehicle_not_my_fuels = {v: list(set(fuels) ^ set(fuel_set[v])) for v in vehicles}

## Define the Objective Function

In [130]:
import re

def get_y_purchase(vehicle_id):
    match = re.search(r'(\d{4})$', vehicle_id)
    if match:
        return int(match.group(1))

get_y_purchase('BEV_S1_2025')

2025

In [131]:
def cardinality(demand):
    num = re.search(r'(\d{1})$', demand)
    if num:
        return int(num.group(1))
cardinality('D1')

1

In [132]:
m = gp.Model("Fleet Decarbonization")
buy = m.addVars(vehicles, vtype=GRB.INTEGER, name="buy") #number of vehicles bought
# use = m.addVars(vehicles, years, vtype=GRB.CONTINUOUS, name="use") 
sell = m.addVars(vehicles, years, vtype=GRB.INTEGER, name="sell")
vehicle_list = m.addVars(vehicles, years, vtype=GRB.INTEGER, name="vehicle_list") #number of vehicles in year y
vehicle_list_fuels = m.addVars(vehicles, years, fuels, distances, vtype=GRB.INTEGER, name="vehicle_list_fuels") #N_v_f
distance = m.addVars(vehicles, years, fuels, distances, vtype=GRB.CONTINUOUS, name="distance")                  

# Objective
m.setObjective(
    gp.quicksum(buy[v] * vehicle_cost[v] for v in vehicles)
    + gp.quicksum(vehicle_cost[v] * resale_value[y - get_y_purchase(v) + 1] * sell[v, y] 
               + vehicle_cost[v] * insurance_cost[y - get_y_purchase(v)+ 1] * vehicle_list[v,y]
               + vehicle_cost[v] * maintenance_cost[y - get_y_purchase(v) + 1] * vehicle_list[v,y]
               for y in years for v in vehicles if ((y >= get_y_purchase(v)) & (y - get_y_purchase(v) <=9)))
    + gp.quicksum(vehicle_list_fuels[v, y, f, d] * distance[v, y, f, d] * fuel_consumption[v, f] * fuel_cost[y, f] 
               for y in years for v in vehicles for f in fuel_set[v] for d in distances),
    GRB.MINIMIZE
)

In [ ]:
m.addConstrs(
    (vehicle_list_fuels[v, y, f, d] == 0 for y in years for f in fuels for d in distances for v in vehicles if distance_bucket_map[vehicle_distance[v]] < distance_bucket_map[d]),
    "Use Variable constraint"
)
m.addConstrs(
    (distance[v, y, f, d] == 0 for f in fuels for y in years for d in distances for v in vehicles if distance_bucket_map[vehicle_distance[v]] < distance_bucket_map[d]),
    "Distance constraint"
)
m.addConstrs(
    (vehicle_list_fuels[v, y, f, d] == 0 for v in vehicles for y in years for f in vehicle_not_my_fuels[v] for d in distances),
    "Use Variable constraint for Fuel Set"
)
m.addConstrs(
    (distance[v, y, f, d] == 0 for v in vehicles for y in years for f in vehicle_not_my_fuels[v] for d in distances),
    "Distance constraint for Fuel Set"
)
m.addConstrs(
    (vehicle_list_fuels[v, y, f, d] == 0 for v in vehicles for y in years for f in fuels for d in distances if ((y < get_y_purchase(v)) | (y - get_y_purchase(v) > 9))),
    "Use Variable constraint for Year Set"
)
m.addConstrs(
    (distance[v, y, f, d] == 0 for v in vehicles for y in years for f in fuels for d in distances if ((y < get_y_purchase(v)) | (y - get_y_purchase(v) > 9))),
    "Distance constraint for Year Set"
)
m.addConstrs(
    (vehicle_list_fuels[v, y, f, d]*vehicle_range[v] >= distance[v, y, f, d] for y in years for f in fuels for d in distances for v in vehicles),
    "Use Variable constraint for Distance"
)
m.addConstrs(
    (gp.quicksum(vehicle_list_fuels[v, y, f, d] * distance[v, y, f, d] * vehile_size_map[v, s] for v in vehicles for f in fuel_set[v]) >= demand[y, s, d] for y in years for s in sizes for d in distances),
    "Demand satisfaction constraint"
)
m.addConstrs(
    (gp.quicksum(vehicle_list_fuels[v, y, f, d] * distance[v, y, f, d] * fuel_consumption[v, f] * fuel_emissions[y, f] for v in vehicles for f in fuel_set[v] for d in distances) <= carbon_budget[y] for y in years),
    "Carbon Emission Satisfaction constraint"
)
m.addConstrs(
    (gp.quicksum(sell[v, y] for y in years if ((y <= get_y_purchase(v) - 1) | (y - get_y_purchase(v) >= 10))) == 0 for v in vehicles),
    "Selling constraint"
)
m.addConstrs(
    (gp.quicksum(sell[v, y] for y in range(2023,2030) if ((y >= get_y_purchase(v)) & (y - get_y_purchase(v) <= 9))) == buy[v] for v in vehicles),
    "Vehicle Mass Balance for 10 Years"
)
m.addConstrs(
    (gp.quicksum(sell[v, y] for y in range(2030,2039) if ((y >= get_y_purchase(v)) & (y - get_y_purchase(v) <= 9))) == buy[v] for v in vehicles),
    "Vehicle Mass Balance for less than 10 Years"
)
m.addConstrs(
    ((gp.quicksum(sell[v, y] for v in vehicles)) <= 0.2 * (gp.quicksum(vehicle_list[v, y] for v in vehicles)) for y in years),
    "Selling Yearly Limit"
)
m.addConstrs(
    (buy[v]>=0 for v in vehicles),
    "Buy >= 0"
)
m.addConstrs(
    (sell[v, y]>=0 for y in years for v in vehicles),
    "Selling >=0"
)
m.addConstrs(
    (vehicle_list_fuels[v, y, f, d]>=0 for y in years for v in vehicles for d in distances for f in fuels),
    "Using List >=0"
)
m.addConstrs(
    (distance[v, y, f, d]>=0 for v in vehicles for y in years for d in distances for f in fuels),
    "Distance List >=0"
)
m.addConstrs(
    (vehicle_list[v, y]>=0 for y in years for v in vehicles),
    "Number of Vehicles >=0"
)
# m.addConstrs(
#     ((buy[v] - (gp.quicksum(sell[v, y] for y in years if ((y >= get_y_purchase(v)) & (y <= j - 1))))) == vehicle_list[v, j] for v in vehicles for j in years),
#     "Buy Sell Equality"
# )
m.addConstrs(
    (vehicle_list[v, y]>=0 for y in years for v in vehicles),
    "Number of Vehicles >=0"
)
m.addConstrs(
    (gp.quicksum(distance[v, y, f, d] for f in fuels for d in distances) <= vehicle_range[v] for v in vehicles for y in years),
    "Yearly Range"
)
m.addConstrs(
    (gp.quicksum(vehicle_list_fuels[v, y, f, d] for f in fuel_set[v] for d in distances) <= vehicle_list[v, y] for v in vehicles for y in years),
    "Additonal Use Balance"
)

In [ ]:
m.optimize()

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-12700, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 361392 rows, 129216 columns and 509188 nonzeros
Model fingerprint: 0x8ba443fd
Model has 20480 quadratic objective terms
Model has 272 quadratic constraints
Variable types: 61440 continuous, 67776 integer (0 binary)
Coefficient statistics:
  Matrix range     [2e-01, 1e+05]
  QMatrix range    [6e-02, 1e+00]
  Objective range  [5e+03, 4e+05]
  QObjective range [2e-01, 8e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [7e+04, 1e+05]
  QRHS range       [2e+03, 1e+07]
Presolve removed 351544 rows and 87824 columns
Presolve time: 0.14s
Presolved: 44089 rows, 49886 columns, 109625 nonzeros
Presolved model has 8492 bilinear constraint(s)
         in product terms.
         Presolve was not able to compute smaller bounds for 

In [112]:
obj = m.getObjective()
print(obj.getValue())

AttributeError: Unable to retrieve attribute 'x'

In [96]:
for v in m.getVars():
    if(v.X != 0):
        print(f"{v.VarName} = {v.X}")

vehicle_list_fuels[BEV_S1_2023,2024,Electricity,D1] = 9.0
vehicle_list_fuels[BEV_S1_2023,2025,Electricity,D1] = 9.0
vehicle_list_fuels[BEV_S1_2023,2026,Electricity,D1] = 18.0
vehicle_list_fuels[BEV_S1_2026,2026,Electricity,D2] = 58.0
vehicle_list_fuels[BEV_S1_2027,2027,Electricity,D1] = 20.0
vehicle_list_fuels[BEV_S1_2027,2027,Electricity,D2] = 60.0
vehicle_list_fuels[BEV_S1_2027,2028,Electricity,D1] = 20.0
vehicle_list_fuels[BEV_S1_2027,2028,Electricity,D2] = 62.0
vehicle_list_fuels[BEV_S1_2027,2029,Electricity,D1] = 10.0
vehicle_list_fuels[BEV_S1_2027,2030,Electricity,D1] = 20.0
vehicle_list_fuels[BEV_S1_2027,2030,Electricity,D2] = 58.0
vehicle_list_fuels[BEV_S1_2027,2031,Electricity,D1] = 22.0
vehicle_list_fuels[BEV_S1_2027,2031,Electricity,D2] = 33.0
vehicle_list_fuels[BEV_S1_2027,2032,Electricity,D1] = 982.0
vehicle_list_fuels[BEV_S1_2027,2032,Electricity,D2] = 34.0
vehicle_list_fuels[BEV_S1_2027,2033,Electricity,D1] = 11.0
vehicle_list_fuels[BEV_S1_2027,2034,Electricity,D1] = 11.

In [119]:
m.computeIIS()

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-12700, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 20 logical processors, using up to 20 threads

IIS computation: initial model status unknown, solving to determine model status
Presolve removed 370364 rows and 86552 columns
Presolve time: 0.08s
Presolved: 30852 rows, 47625 columns, 77792 nonzeros
Presolved model has 4960 bilinear constraint(s)
         in product terms.
         Presolve was not able to compute smaller bounds for these variables.
         Consider bounding these variables or reformulating the model.


Solving non-convex MIQCP

Variable types: 25441 continuous, 22184 integer (0 binary)

Explored 1 nodes (0 simplex iterations) in 0.23 seconds (0.17 work units)
Thread count was 20 (of 20 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -

Computing Irreducible Inconsistent Subsyst

In [120]:
m.write("model.ilp")
for constr in m.getConstrs():
    if constr.IISConstr:
        print(f"Constraint {constr.ConstrName} is in the IIS")
        # Print the linear expression of the constraint
        lhs = m.getRow(constr)
        rhs = constr.getAttr(GRB.Attr.RHS)
        sense = constr.Sense
        print(f"  LHS: {lhs}")
        print(f"  RHS: {rhs}")
        print(f"  Sense: {sense}")

Constraint Use Variable constraint[2023,Electricity,D3,BEV_S3_2023] is in the IIS
  LHS: vehicle_list_fuels[BEV_S3_2023,2023,Electricity,D3]
  RHS: 0.0
  Sense: =
Constraint Use Variable constraint[2023,Electricity,D3,BEV_S3_2024] is in the IIS
  LHS: vehicle_list_fuels[BEV_S3_2024,2023,Electricity,D3]
  RHS: 0.0
  Sense: =
Constraint Use Variable constraint[2023,Electricity,D3,BEV_S3_2028] is in the IIS
  LHS: vehicle_list_fuels[BEV_S3_2028,2023,Electricity,D3]
  RHS: 0.0
  Sense: =
Constraint Distance constraint[Electricity,2023,D3,BEV_S3_2025] is in the IIS
  LHS: distance[BEV_S3_2025,2023,Electricity,D3]
  RHS: 0.0
  Sense: =
Constraint Distance constraint[Electricity,2023,D3,BEV_S3_2026] is in the IIS
  LHS: distance[BEV_S3_2026,2023,Electricity,D3]
  RHS: 0.0
  Sense: =
Constraint Distance constraint[Electricity,2023,D3,BEV_S3_2027] is in the IIS
  LHS: distance[BEV_S3_2027,2023,Electricity,D3]
  RHS: 0.0
  Sense: =
Constraint Use Variable constraint for Year Set[BEV_S3_2029,2023

In [16]:
vehicle_distance['BEV_S2_2028']

'D2'